In [78]:
import pandas as pd
import re
from datetime import datetime
df = pd.read_csv("./IMDb 2024 Movies TV Shows.csv")

# converts foreign currencies to usd, replaces unknown budgets with None
def convert_budget_currency(budget_string):
    if budget_string == '000':
        return None
    rate = 1
    if '€' in budget_string:
        rate = 1.08
    elif 'R$' in budget_string:
        rate = 0.17364712
    elif '₹' in budget_string:
        rate = 0.012
    elif 'HK$' in budget_string:
        rate = 0.13
    elif '$' not in budget_string:
        return None
    else:
        return budget_string[1:].replace(',', '')
    starting_index = re.search(r'\d', budget_string).start()
    return rate * int(budget_string[starting_index:].replace(',', ''))

# converts Run_Time to integer minutes, or None if not found
def convert_time(time_string):
    if time_string == '[\'Not_Found\']':
        return None
    parts = time_string.split()
    minutes = 0
    for i in range(1, len(parts)):
        if 'hour' in parts[i]:
            minutes += int(parts[i - 1][2:]) * 60
        elif 'minute' in parts[i]:
            minutes += int(parts[i - 1].replace('\'', '').replace('[', ''))
    return minutes

# creates column for release country
def release_country(release_date_string):
    if (release_date_string == '[]'):
        return None
    country_index = release_date_string.index('(')
    return release_date_string[country_index + 1: -3]

# converts release date to format year-month-day, None if not available
def convert_release_date(release_date_string):
    if (release_date_string == '[]'):
        return None
    country_index = release_date_string.index('(')
    date = datetime.strptime(release_date_string[2:country_index - 1], '%B %d, %Y')
    return date.strftime('%Y-%m-%d')

# converts revenue to integer value, None if not available
def convert_revenue(revenue_string):
    if revenue_string == '000':
        return None
    return revenue_string.replace('$','').replace(',','')
# converts vote count to ints
def convert_vote_count(vote_count_string):
    if '.' in vote_count_string:
        parts = vote_count_string.split('.')
        return int(parts[0]) * 1000 + int(parts[1][:-1]) * 100
    thousands = re.match(r'\d+K', vote_count_string)
    if thousands:
        return int(vote_count_string[:-1]) * 1000
    return int(vote_count_string)

# removes Not_Found values from Storyline column
def storyline(storyline_string):
    if storyline_string == 'Not_Found':
        return None
    return storyline_string

# removes Not_Found values from Tagline column
def tagline(tagline_string):
    if tagline_string == 'Not_Found':
        return None
    return tagline_string

df['Budget'] = df['Budget'].apply(convert_budget_currency)
df['Run_Time'] = df['Run_Time'].apply(convert_time)
df['Release_Country'] = df['Release_Date'].apply(release_country)
df['Release_Date'] = df['Release_Date'].apply(convert_release_date)
df['Revenue'] = df['Revenue'].apply(convert_revenue)
df['Vote_Count'] = df['Vote_Count'].apply(convert_vote_count)
df['Tagline'] = df['Tagline'].apply(tagline)
df['Storyline'] = df['Storyline'].apply(storyline)
df.to_csv('./Processed IMDb 2024 Movies TV Shows.csv')

PermissionError: [Errno 13] Permission denied: './Processed IMDb 2024 Movies TV Shows.csv'